In [1]:
from sql_connection import sql_connect
from sql_connection import write_sql_conn_create
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

### Reading the data from SQL Server 

In [2]:
query = 'select * from dbo.application_with_history'

In [3]:
app_data = sql_connect(query)
app_data.head()

Connection to SQL Server established successfully.
Connection closed.


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVERAGE_EXTERNAL_RATING,TOT_PREV_APP,PREV_APPROVED_CNT,PREV_CANCELLED_CNT,PREV_REFUSED_CNT,PREV_UNUSED_CNT
0,100002,1,Cash loans,M,0,1,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120.0,-1,1,1,0,1,1,0,Laborers,1,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,2,2,2,2,-1134,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.1618,0,0,0,0,0
1,100003,0,Cash loans,F,0,0,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291.0,-1,1,1,0,1,1,0,Core staff,2,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,1,0,1,0,-828,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.4668,0,0,0,0,0
2,100004,0,Revolving loans,M,1,1,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531.0,26,1,1,1,1,1,0,Laborers,1,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,0,0,0,0,-815,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.6427,0,0,0,0,0
3,100006,0,Cash loans,F,0,1,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437.0,-1,1,1,0,1,0,0,Laborers,2,2,2,WEDNESDAY,17,0,0,0,0,0,0,Business Entity Type 3,NaN,0.650442,NaN,2,0,2,0,-617,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.6504,0,0,0,0,0
4,100007,0,Cash loans,M,0,1,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458.0,-1,1,1,0,1,0,0,Core staff,1,2,2,THURSDAY,11,0,0,0,0,1,1,Religion,NaN,0.322738,NaN,0,0,0,0,-1106,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.3227,1,1,0,0,0


In [4]:
app_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 81 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   SK_ID_CURR                   49999 non-null  int64  
 1   TARGET                       49999 non-null  int64  
 2   NAME_CONTRACT_TYPE           49999 non-null  object 
 3   CODE_GENDER                  49999 non-null  object 
 4   FLAG_OWN_CAR                 49999 non-null  int64  
 5   FLAG_OWN_REALTY              49999 non-null  int64  
 6   CNT_CHILDREN                 49999 non-null  int64  
 7   AMT_INCOME_TOTAL             49999 non-null  float64
 8   AMT_CREDIT                   49999 non-null  float64
 9   AMT_ANNUITY                  49998 non-null  float64
 10  AMT_GOODS_PRICE              49910 non-null  float64
 11  NAME_TYPE_SUITE              49999 non-null  object 
 12  NAME_INCOME_TYPE             49999 non-null  object 
 13  NAME_EDUCATION_T

The dataset contains many categorical text features that must be converted into numerical format before feeding them into machine learning models. We use **One-Hot Encoding** to handle nominal categorical variables.

Approach:
1. Identify all object or category type columns.
2. Apply one-hot encoding using `pd.get_dummies()`.
3. Ensure numerical columns remain unchanged.
4. Drop the first level to avoid multicollinearity.

In [5]:
#str_col = app_data.select_dtypes('object').columns
#app_data_encoded = pd.get_dummies(app_data, columns = str_col, drop_first=True)

##  Handle Missing Values

While most of the data cleaning was done in SQL, the exported dataset still contains **some missing values**, which must be handled before applying machine learning models.

###  Why Handle Missing Values?
- Many ML models **cannot process NaN values**.
- Missing values can lead to **biased results or errors** during training.
- The handling strategy depends on the **data type** and **business context** of each feature.

In [6]:
percentage_missing = pd.DataFrame(app_data.isnull().sum()/len(app_data), columns=['Missing'])
percentage_missing[percentage_missing['Missing'] > 0]

,Missing
AMT_ANNUITY,0.000020
AMT_GOODS_PRICE,0.001780
DAYS_REGISTRATION,0.000160
DAYS_ID_PUBLISH,0.000060
EXT_SOURCE_1,0.563451
EXT_SOURCE_2,0.002640
EXT_SOURCE_3,0.198884
AMT_REQ_CREDIT_BUREAU_HOUR,0.134683
AMT_REQ_CREDIT_BUREAU_DAY,0.134683
AMT_REQ_CREDIT_BUREAU_WEEK,0.134683


## Amount Annuity

- We assume that **20% of a person's monthly income** is allocated toward repaying credit.
- Given:
  - **Total Credit Given/Used** = 450,000
  - **Annual Income** = 180,000
  - **Monthly Repayment Capacity** = 20% of (180,000 / 12) = 3,000
  - Or, directly: 20% of 180,000 = 36,000 per year → 3,000 per month

In [7]:
app_data[app_data['AMT_ANNUITY'].isna()]

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVERAGE_EXTERNAL_RATING,TOT_PREV_APP,PREV_APPROVED_CNT,PREV_CANCELLED_CNT,PREV_REFUSED_CNT,PREV_UNUSED_CNT
47531,155054,0,Cash loans,M,0,0,0,180000.0,450000.0,NaN,450000.0,Unaccompanied,Commercial associate,Incomplete higher,Single / not married,House / apartment,0.026392,-10668,-2523,-4946.0,-3238.0,-1,1,1,1,1,1,0,High skill tech staff,1,2,2,WEDNESDAY,13,0,0,0,0,0,0,Business Entity Type 3,NaN,0.626896,0.372334,1,0,1,0,-2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,0.4996,0,0,0,0,0


In [8]:
app_data.loc[app_data['AMT_ANNUITY'].isna(), 'AMT_ANNUITY'] = \
    0.2 * app_data.loc[app_data['AMT_ANNUITY'].isna(), 'AMT_INCOME_TOTAL']

In cases where `AMT_GOODS_PRICE` is missing or not available, we assume:

>  `AMT_GOODS_PRICE ≈ AMT_CREDIT`

This is a reasonable assumption in consumer finance where the **credit issued closely matches the purchase price** of the goods.


In [9]:
app_data.loc[app_data['AMT_GOODS_PRICE'].isna(), 'AMT_GOODS_PRICE'] = \
    app_data.loc[app_data['AMT_GOODS_PRICE'].isna(), 'AMT_CREDIT']

We impute missing values in the following columns using the **median** of each:

- `DAYS_ID_PUBLISH`
- `DAYS_REGISTRATION`

This ensures no missing values remain in these important date-related features before modeling.


In [10]:
app_data['DAYS_ID_PUBLISH'] = app_data['DAYS_ID_PUBLISH'].fillna(app_data['DAYS_ID_PUBLISH'].median())
app_data['DAYS_REGISTRATION'] = app_data['DAYS_REGISTRATION'].fillna(app_data['DAYS_REGISTRATION'].median())



The dataset originally includes three external rating features:

- `EXT_SOURCE_1`
- `EXT_SOURCE_2`
- `EXT_SOURCE_3`

However, during the **SQL preprocessing stage**, we created a new feature:  
**`AVERAGE_EXTERNAL_RATING`** — the average of all available EXT_SOURCE values.

In [11]:
app_data.drop(columns=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'], inplace=True)

In [12]:
app_data['AVERAGE_EXTERNAL_RATING'] = \
    app_data['AVERAGE_EXTERNAL_RATING'].fillna(app_data['AVERAGE_EXTERNAL_RATING'].median())

In [13]:
app_data['ISMISSING_BUREAU_INFO'] = \
    app_data['AMT_REQ_CREDIT_BUREAU_DAY'].apply(lambda x: 1 if pd.isna(x) else 0)

### Create a Binary Missingness Indicator
We define a new feature:
> `ISMISSING_BUREAU_INFO = 1` if `AMT_REQ_CREDIT_BUREAU_DAY` is missing, else `0`

This allows models to treat **missing data as a signal**, potentially identifying applicants with no prior bureau history.

---

###  Impute Missing Bureau Features with 0

For the following bureau-related columns, we assume that missing values imply **no recorded activity**, and thus we fill them with `0`:

- `AMT_REQ_CREDIT_BUREAU_HOUR`
- `AMT_REQ_CREDIT_BUREAU_DAY`
- `AMT_REQ_CREDIT_BUREAU_WEEK`
- `AMT_REQ_CREDIT_BUREAU_MON`
- `AMT_REQ_CREDIT_BUREAU_QRT`
- `AMT_REQ_CREDIT_BUREAU_YEAR`

In [14]:
app_data['ISMISSING_BUREAU_INFO'] = \
    app_data['AMT_REQ_CREDIT_BUREAU_DAY'].apply(lambda x: 1 if pd.isna(x) else 0)

In [15]:
col = ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']
app_data[col] = app_data[col].fillna(0)

In [16]:
percentage_missing = pd.DataFrame(app_data.isnull().sum()/len(app_data), columns=['Missing'])
percentage_missing[percentage_missing['Missing'] > 0]

,Missing


### Now no missing values, and storing this data in sql

In [17]:
bool_cols = app_data.select_dtypes("bool").columns
app_data[bool_cols] = app_data[bool_cols].astype(int)

In [18]:
app_data.dtypes

SK_ID_CURR                       int64
TARGET                           int64
NAME_CONTRACT_TYPE              object
CODE_GENDER                     object
FLAG_OWN_CAR                     int64
FLAG_OWN_REALTY                  int64
CNT_CHILDREN                     int64
AMT_INCOME_TOTAL               float64
AMT_CREDIT                     float64
AMT_ANNUITY                    float64
AMT_GOODS_PRICE                float64
NAME_TYPE_SUITE                 object
NAME_INCOME_TYPE                object
NAME_EDUCATION_TYPE             object
NAME_FAMILY_STATUS              object
NAME_HOUSING_TYPE               object
REGION_POPULATION_RELATIVE     float64
DAYS_BIRTH                       int64
DAYS_EMPLOYED                    int64
DAYS_REGISTRATION              float64
DAYS_ID_PUBLISH                float64
OWN_CAR_AGE                      int64
FLAG_MOBIL                       int64
FLAG_EMP_PHONE                   int64
FLAG_WORK_PHONE                  int64
FLAG_CONT_MOBILE         

In [19]:
engine = write_sql_conn_create()
app_data.to_sql('cleaned_application_ml', engine, if_exists='replace', index=False)

Connection to SQL Server established successfully.


1